<a href="https://colab.research.google.com/github/itc202/praktik/blob/main/RECOGNITION_CATS_and_DOGS_VGG_from_Kaggle_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание кошек и собак

Источник:
https://www.kaggle.com/code/scratchpad/notebook7b26701e34/edit

Все ноутбуки на эту тему:
https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/code



In [ ]:
import os
import random

import cv2
import numpy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

import tensorflow
from tensorflow.keras.applications import VGG16

Дата сет можно скачать здесь:
https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [ ]:
#В своем ноутбуке нужно пропсать адреса своих папок с данными
CATS = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat"
DOGS = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog"

IMAGE_SIZE = (64, 64)

In [ ]:
def get_data(directory: str, label: int) -> numpy.ndarray:
    images = []
    labels = []

    for file_name in os.listdir(directory):
        image = cv2.imread(os.path.join(directory, file_name))

        if image is not None:
            image = image.astype(numpy.float32) / 255.
            image = cv2.resize(image, IMAGE_SIZE)

            images.append(image)
            labels.append(label)

    return numpy.array([(image, label) for image, label in zip(images, labels)], dtype=object)

In [ ]:
raw_data = numpy.concatenate((get_data(CATS, 0), get_data(DOGS, 1)), axis=0)

In [ ]:
data, labels = numpy.array([pair[0] for pair in raw_data]), numpy.array([pair[1] for pair in raw_data])

In [ ]:
one_hot_encoder = OneHotEncoder()

labels = one_hot_encoder.fit_transform(labels.reshape(-1, 1))

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels.toarray(), test_size=0.2, random_state=1)

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
vgg16.trainable = False

In [ ]:
model = Sequential([
    vgg16,
    Flatten(),
    Dense(2048, activation="relu"),
    Dropout(0.25),
    Dense(1024, activation="relu"),
    Dropout(0.25),
    Dense(512, activation="relu"),
    Dropout(0.25),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(2, activation="softmax"),
])

In [ ]:
model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
)

In [ ]:
#model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

In [ ]:
history = model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

Далее можно всатвить задание и код из исходного ноутбука , где созадем свою сеть с нуля

In [ ]:
# Построим изменение точности на трейновой (оранжевая линия) и тестовой (синяя линия)
# выборках
plt.plot(history.history['val_accuracy'], '-o', label='validation accuracy')
plt.plot(history.history['accuracy'], '--s', label='training accuracy')
plt.legend();

In [ ]:
# Построим изменение потерь на трейновой (оранжевая линия) и тестовой (синяя линия)
# выборках
plt.plot(history.history['val_loss'], '-o', label='validation loss')
plt.plot(history.history['loss'], '--s', label='training loss')
plt.legend();